In [ ]:
pip install youtube-transcript-api transformers gradio torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [ ]:
!pip install gradio torch

In [ ]:
import gradio as gr
from youtube_transcript_api import YouTubeTranscriptApi
from transformers import pipeline

# Summarization pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Question Answering pipeline
qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")

def get_transcript(video_url):
    """Extracts transcript from a YouTube video URL"""
    try:
        video_id = video_url.split("v=")[-1].split("&")[0]
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        full_text = " ".join([entry["text"] for entry in transcript])
        return full_text
    except Exception as e:
        return f"Error: {str(e)}"

def summarize_transcript(video_url):
    """Summarizes the extracted transcript"""
    transcript = get_transcript(video_url)
    if transcript.startswith("Error:"):
        return transcript, ""

    # Process transcript in chunks of 1000 characters
    chunks = [transcript[i:i+1000] for i in range(0, len(transcript), 1000)]
    summary = ""
    for chunk in chunks:
        sum_part = summarizer(chunk, max_length=150, min_length=40, do_sample=False)[0]['summary_text']
        summary += sum_part + " "

    return transcript, summary.strip()

def answer_question(question, summary_text):
    """Answers user questions based on the summarized video content"""
    if not summary_text or summary_text.startswith("Error:"):
        return "Please summarize a video first!"

    answer = qa_pipeline(question=question, context=summary_text)
    return answer["answer"]

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## 🎥 YouTube Video Summarizer & Q&A Chatbot")

    video_url_input = gr.Textbox(label="YouTube Video URL")
    transcript_output = gr.Textbox(label="Transcript", lines=10)
    summary_output = gr.Textbox(label="Summary", lines=5)

    summarize_btn = gr.Button("Summarize Video")
    summarize_btn.click(summarize_transcript, inputs=[video_url_input], outputs=[transcript_output, summary_output])

    gr.Markdown("### ❓ Ask Questions About the Video")
    chatbot_ui = gr.Chatbot()
    user_msg = gr.Textbox(label="Your Question")
    chat_state = gr.State([])

    def chatbot_response(message, chat_history, summary_text):
        """Handles user queries based on video summary"""
        response = answer_question(message, summary_text)
        chat_history.append((message, response))
        return chat_history, chat_history

    user_msg.submit(chatbot_response, inputs=[user_msg, chat_state, summary_output], outputs=[chatbot_ui, chat_state])

demo.launch()


Device set to use cpu
Device set to use cpu
<ipython-input-11-cd4a0eb8d028>:56: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot_ui = gr.Chatbot()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ef6b6d1d7b5f145e89.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
